# Credit Risk Ensemble

### Part Zero - Initial Imports

In [1]:
# Import a library to ignore the warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import the libraries I will need for this project 
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import os 

# Identify the directory I am working in 
os.getcwd()

'/Users/cg/Documents/FinTech_Bootcamp/[Unit_11]Machine_Learning/[Homework]Risky_Buisness'

### Part One -  Read the CSV and Perform Basic Data Cleaning

In [11]:
# Load the data for the project
csv_path = Path('/Users/cg/Documents/FinTech_Bootcamp/[Unit_11]Machine_Learning/[Homework]Risky_Buisness/LoanStats_2019Q1.csv')
df = pd.read_csv(csv_path)

# Preview the data
df.head(10)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
5,25000.0,0.1797,634.43,MORTGAGE,90000.0,Source Verified,Mar-2019,low_risk,n,18.49,...,93.1,66.7,0.0,0.0,550717.0,418934.0,18800.0,369051.0,N,N
6,20400.0,0.2000,540.48,RENT,51315.0,Source Verified,Mar-2019,low_risk,n,25.63,...,100.0,0.0,1.0,0.0,40023.0,23469.0,3500.0,27523.0,N,N
7,10000.0,0.1557,349.46,OWN,66000.0,Source Verified,Mar-2019,low_risk,n,9.00,...,100.0,16.7,0.0,0.0,29300.0,22377.0,8400.0,12500.0,N,N
8,14000.0,0.0881,443.96,RENT,45000.0,Not Verified,Mar-2019,low_risk,n,10.24,...,100.0,0.0,0.0,0.0,25100.0,9724.0,12100.0,3000.0,N,N
9,10000.0,0.1474,345.39,RENT,63295.0,Verified,Mar-2019,low_risk,n,39.34,...,97.2,71.4,0.0,0.0,119822.0,102924.0,15300.0,80722.0,N,N


### Part Two - Split the Data into Training and Testing

In [15]:
# View all the columns in the df
print(df.columns)

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

In [17]:
# Here I will make the features -  Drop columns with only a single value ordates
x = df.drop(columns=[
'next_pymnt_d',
'acc_now_delinq',
'num_tl_120dpd_2m',
'num_tl_30dpd',
'tax_liens',
'hardship_flag',
'debt_settlement_flag',
'loan_status',
'issue_d',
'pymnt_plan',
'recoveries',
'collection_recovery_fee',
])

In [19]:
# Set columns with string values to have binary values
x_binary = pd.get_dummies(X, columns=[
    'application_type', 
    'initial_list_status',
    'home_ownership',
    'verification_status',
    ])

In [20]:
# Define and create the target
y = df['loan_status']

In [21]:
x.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,12.571995,0.052138,2.219423,95.057627,30.626217,0.125972,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,6.021318,0.390633,1.897432,8.326426,33.631463,0.336732,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,20.000000,0.000000,0.000000,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,8.000000,0.000000,1.000000,93.000000,0.000000,0.000000,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,11.000000,0.000000,2.000000,100.000000,20.000000,0.000000,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,16.000000,0.000000,3.000000,100.000000,50.000000,0.000000,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,72.000000,18.000000,19.000000,100.000000,100.000000,4.000000,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [23]:
# Check the balance of the target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [24]:
# Import the library I need to perform the train/test split 
from sklearn.model_selection import train_test_split

# Split the X and y into X_train, X_test, y_train, y_test
x_train, x_test, y_train, y_test= train_test_split(x_binary, y, random_state=1, stratify=y)

# Display the shape of the x-train set
x_train.shape

(51612, 81)

### Part Three - Data Pre-Processing

##### Scale the training and testing data using the ***StandardScaler*** from ***SciKit-Learn***. Remember that when scaling the data, you only scale the features data (*X_train* and *X_testing*).

In [25]:
# Import the library needed to perform the scaler
from sklearn.preprocessing import StandardScaler

# Create an instance of standard scaler
scaler = StandardScaler()

In [27]:
# Fit the Standard Scaler with the training data - only train the test set
x_scaler = scaler.fit(x_train)

In [31]:
# Scale the training and testing data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

### Part Four - Ensemble Learners

##### In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the ***imbalanced_classification_report*** from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score


#### Balanced Random Forest Classifier

In [32]:
# Import the library needed for the RFC
from sklearn.ensemble import RandomForestClassifier

# Resample the training data with the BalancedRandomForestClassifier
rfc_model = RandomForestClassifier(n_estimators=100, random_state=1)

# Fit the model
rfc_model = rfc_model.fit(x_train_scaled, y_train)

In [34]:
# Let the model make predictions
rfc_y_predictions = rfc_model.predict(x_test_scaled)

# Calculate the balanced accuracy score
rfc_bas = balanced_accuracy_score(y_test, rfc_y_predictions)
print(f" The Balanced Accuracy Score is : {rfc_bas}")

 The Balanced Accuracy Score is : 0.6662869494099778


In [35]:
# Create the confusion matrix
rfc_confusion_matrix = confusion_matrix(y_test, rfc_y_predictions)

# Save the data to a df 
rfc_confusion_matrix_df = pd.DataFrame(
    rfc_confusion_matrix, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"])

# Display the confusion matrix results
display(rfc_confusion_matrix_df)

,Predicted 0,Predicted 1
Actual 0,29,58
Actual 1,13,17105


In [36]:
# Print the imbalanced classification report
print('This is the Classification Report:')
...
print("")
...
print(classification_report_imbalanced(y_test, rfc_y_predictions))

This is the Classification Report:

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.69      0.33      1.00      0.45      0.58      0.31        87
   low_risk       1.00      1.00      0.33      1.00      0.58      0.36     17118

avg / total       1.00      1.00      0.34      1.00      0.58      0.36     17205



In [57]:
# Save the importances of the features
importances = rfc_model.feature_importances_

# List the features sorted in descending order by feature importance
features = sorted(zip(importances, X.columns), reverse=True)
features

[(0.07473152937005441, 'total_pymnt'),
 (0.06798914569455675, 'total_rec_prncp'),
 (0.06757798201480475, 'out_prncp'),
 (0.062377427139989866, 'initial_list_status'),
 (0.05874587198969456, 'out_prncp_inv'),
 (0.019902614170432657, 'bc_util'),
 (0.018092067662149207, 'total_acc'),
 (0.017414910104833602, 'inq_last_12m'),
 (0.01694012493225354, 'annual_inc'),
 (0.016567489739830955, 'bc_open_to_buy'),
 (0.016220021076255756, 'total_cu_tl'),
 (0.01599439539451493, 'total_bal_il'),
 (0.015834366180569732, 'installment'),
 (0.015763000270933702, 'pct_tl_nvr_dlq'),
 (0.015456438989875292, 'num_tl_op_past_12m'),
 (0.01468462665498181, 'collections_12_mths_ex_med'),
 (0.014565224467175994, 'revol_bal'),
 (0.014530285037317014, 'total_pymnt_inv'),
 (0.014330739356450985, 'inq_fi'),
 (0.014209851829314488, 'pub_rec_bankruptcies'),
 (0.01375222298689567, 'open_act_il'),
 (0.013705210118694128, 'percent_bc_gt_75'),
 (0.01362925956879834, 'home_ownership'),
 (0.013031025208187687, 'open_acc'),
 (0

#### Easy Ensemble Classifier

In [39]:
# Import the library needed for the classifier 
from imblearn.ensemble import EasyEnsembleClassifier

# Train the Classifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)

# Fit the model 
eec_model = eec_model.fit(x_train_scaled, y_train)

In [45]:
# Let the model make predictions
eec_y_predictions = eec_model.predict(x_test_scaled)

# Calculate the balanced accuracy score
eec_bas = balanced_accuracy_score(y_test, eec_y_predictions)

# Display the BAS
print(f"The Balanced Accuracy Score is: {eec_bas}")

The Balanced Accuracy Score is: 0.7504975605432476


In [41]:
# Create the confusion matrix
eec_confusion_matrix = confusion_matrix(y_test, eec_y_predictions)

# Save the data to a df
eec_confusion_matrix_df = pd.DataFrame(
    eec_confusion_matrix, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"])

# Show the results
display(eec_confusion_matrix_df)

,Predicted 0,Predicted 1
Actual 0,62,25
Actual 1,3623,13495


In [53]:
# Show the Classification Report
print("This is the Classification Report:")
...
print("")
...
print(classification_report_imbalanced(y_test, eec_y_predictions))

This is the Classification Report:

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.71      0.79      0.03      0.75      0.56        87
   low_risk       1.00      0.79      0.71      0.88      0.75      0.57     17118

avg / total       0.99      0.79      0.71      0.88      0.75      0.57     17205



### Part Five - Final Questions

#### Which model had the best balanced accuracy score?

In [54]:
# Hide the input cell
{
    "tags": [
        "hide-input",
    ]
}

# View the the BAS scores for the different models 
print("These are the Balanced Accuracy Scores:")
print("")
print(f"The Balanced Random Forest Classifier had a BAS of: {rfc_bas}")
print(f"The Easy Ensemble Classifier had a BAS of: {eec_bas}")

These are the Balanced Accuracy Scores:

The Balanced Random Forest Classifier had a BAS of: 0.6662869494099778
The Easy Ensemble Classifier had a BAS of: 0.7504975605432476


From the results I can tell that the **Easy Ensemble Classifier** had a higher BAS with a score of **0.7505**.

#### Which model had the best recall score?

The ***Random Forest Classifier*** had the best recall score with a value of **1.0**. The opposition, *Easy Ensemble Classifier*, had a recall score of *0.79*. Both of these scores can be found in there Classification Reports in the row and column *avg/total* & *rec*, respecfully.

#### Which model had the best geographical mean score?

The ***Easy Ensemble Classifier*** had the best recall score with a value of **0.75**. The opposition, *Random Forest Classifier*, had a recall score of *0.58*. Both of these scores can be found in there Classification Reports in the row and column *avg/total* & *geo*, respecfully.

#### What were the top three features?

In [62]:
# Hide the input cell
{
    "tags": [
        "hide-input",
    ]
}

# View the top three features
features[0:3]

[(0.07473152937005441, 'total_pymnt'),
 (0.06798914569455675, 'total_rec_prncp'),
 (0.06757798201480475, 'out_prncp')]

Here I can see that the top three features were *Total Payment, *Total Recieved Principle (Priciple recieved to date)*, & *Outstanding Priciple (Remaining outstanding principle for total amount funded)*. I know these are the top three because it is the first three records of my sorted variable *features*.